In [ ]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import numpy as np

In [ ]:
import requests
import csv

url = "YOUR_URL_HERE"
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
}

def fetch_reviews(params):
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        return data["result"]["productReviews"]["content"]
    else:
        print(f"Failed to retrieve reviews for contentId {params['contentId']} page {params['page']}")
        return []

# Parameters for the product
params = {
    "sellerId": "238172",
    "contentId": "143581360",
    "page": "1",
    "order": "DESC",
    "orderBy": "Score",
    "channelId": "1"
}

all_reviews = []
for page in range(1, 3):  # Fetch reviews from pages 1 and 2
    params["page"] = str(page)
    reviews = fetch_reviews(params)
    all_reviews.extend(reviews)

# CSV file name based on contentId
csv_file = f'reviews_{params["contentId"]}.csv'

fieldnames = ["User", "Rating", "Comment", "Date", "Seller", "Likes"]
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for review in all_reviews:
        writer.writerow({
            "User": review.get('userFullName', ''),  
            "Rating": review.get('rate', ''),  
            "Comment": review.get('comment', ''),
            "Date": review.get('commentDateISOtype', ''),
            "Seller": review.get('sellerName', ''),
            "Likes": review.get('reviewLikeCount', 0)
        })

print(f'Data has been written to {csv_file}')

In [ ]:
import requests
import csv

url = "YOUR_URL_HERE"
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
}

# Parametreler
params = {
    "sellerId": "238172",
    "contentId": "143581360",
    "page": "1",
    "order": "DESC",
    "orderBy": "Score",
    "channelId": "1"
}

def fetch_reviews(params):
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  # HTTP hatalarını kontrol etmek için
        data = response.json()
        return data["result"]["productReviews"]["content"]
    except (requests.exceptions.RequestException, KeyError, TypeError) as e:
        print(f"Error fetching reviews: {e}")
        return []  # Veya hata durumunda farklı bir değer döndürebilirsiniz


all_comments = []  # Sadece yorumları tutacak liste
page = 1
while True:
    params["page"] = str(page)
    reviews = fetch_reviews(params)
    if not reviews:
        break

    for review in reviews:
        all_comments.append(review['comment'])  # Yorumları listeye ekle

    page += 1

# CSV dosya adı
csv_file = f'yorumlar_{params["contentId"]}.csv'

# Yorumları CSV dosyasına yaz
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for comment in all_comments:
        writer.writerow([comment])

print(f'Yorumlar {csv_file} dosyasına yazıldı.')